In [31]:
import deepchem as dc
tasks, datasets, transformers = dc.molnet.load_muv()
train_dataset, valid_dataset, test_dataset = datasets
train_smiles = train_dataset.ids
valid_smiles = valid_dataset.ids

Loading dataset from disk.
Loading dataset from disk.
Loading dataset from disk.


In [32]:
datasets

(<deepchem.data.datasets.DiskDataset at 0x7f24c0392d68>,
 <deepchem.data.datasets.DiskDataset at 0x7f2570c6ec50>)

In [33]:
train_smiles

array(['Cc1cccc(N2CCN(C(=O)C34CC5CC(CC(C5)C3)C4)CC2)c1C',
       'Cn1ccnc1SCC(=O)Nc1ccc(Oc2ccccc2)cc1',
       'COc1cc2c(cc1NC(=O)CN1C(=O)NC3(CCc4ccccc43)C1=O)oc1ccccc12', ...,
       'CC(C)C(=O)Nc1ccc(NC(=O)c2cccs2)cn1',
       'CCN1/C(=C2\\OC(=S)N(C)C2=O)Sc2ccccc21',
       'COc1ccc(/C=C2\\OC(=S)N(C)C2=O)cc1'], dtype=object)

In [15]:
train_dataset.ids

array(['Cc1cccc(N2CCN(C(=O)C34CC5CC(CC(C5)C3)C4)CC2)c1C',
       'Cn1ccnc1SCC(=O)Nc1ccc(Oc2ccccc2)cc1',
       'COc1cc2c(cc1NC(=O)CN1C(=O)NC3(CCc4ccccc43)C1=O)oc1ccccc12', ...,
       'CC(C)C(=O)Nc1ccc(NC(=O)c2cccs2)cn1',
       'CCN1/C(=C2\\OC(=S)N(C)C2=O)Sc2ccccc21',
       'COc1ccc(/C=C2\\OC(=S)N(C)C2=O)cc1'], dtype=object)

In [16]:
train_dataset.w

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 1., 1.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
train_dataset.y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
tokens = set()
for s in train_smiles:
  tokens = tokens.union(set(c for c in s))
tokens = sorted(list(tokens))

In [19]:

from deepchem.models.tensorgraph.optimizers import Adam, ExponentialDecay
max_length = max(len(s) for s in train_smiles)
model = dc.models.SeqToSeq(tokens,
                           tokens,
                           max_length,
                           encoder_layers=2,
                           decoder_layers=2,
                           embedding_dimension=256,
                           model_dir='fingerprint')
batches_per_epoch = len(train_smiles)/model.batch_size
model.set_optimizer(Adam(learning_rate=ExponentialDecay(0.004, 0.9, batches_per_epoch)))

In [ ]:
def generate_sequences(epochs):
  for i in range(epochs):
    for s in train_smiles:
      yield (s, s)

model.fit_sequences(generate_sequences(1))